# Seventh GF Summer School 2021

----

<img align="left" src="resources/accelerated-text-logo.png" height="100">

Accelerated Text is the natural language generation engine. It will automatically create narratives highlighting outliers, describing trends, and providing explanations in the style and wording most suitable for the intended readers.

### Why low code?

* Closed NLG systems can not react to changing requirements or previously unobserved facts.
* Someone working in a given domain has to be able change the logic defining how data is transformed into narrative.
* Having a domain specific abstraction above the grammar explains how text was produced given a data set. 

### Start the system

First clone the project

```bash
git clone https://github.com/tokenmill/accelerated-text.git
```

Than start the application

```
make run-app
```

Detailed documentation at https://accelerated-text.readthedocs.io/en/latest/installation/

### Elements of Accelerated Text

* Blockly based Visual Programming Language
* Main abstractions:
    * Document Plan
    * Dictionary
    * Abstract Meaning representation
    * Grammatical Framework


<img align="left" src="resources/house_on_the_hill_1_dp.png">

### Interactions via Notebook

In [2]:
from acctext import AcceleratedText

at = AcceleratedText()
at.health()

{'health': 'Ok'}

In [5]:
at.clear_state()
at.restore_state('resources/state.zip')

### Semantic Graph

Sematic graph of the "House on the hill"

<img align="left" src="resources/house_on_the_hill_1.png">

### Abstract Meaning Representation and Domain Grammar

http://localhost:8080/amr

AMR allows to define high level domain specific blocks which can be used by non-linguists.


http://localhost:8080/dlg

This is RGL level where grammar developers work.

### Compext Document Plan and its Semantic Graph



<img align="left" src="resources/complex_plan.png">

## Change plan and interact with AT

Change the plan. Let us say I want to generate BROTHER or SISTER

In [11]:
at.create_dictionary_item('sister', 'N', ['sister'])

{'id': 'sister_N_Eng',
 'key': 'sister',
 'forms': ['sister'],
 'category': 'N',
 'language': 'Eng',
 'attributes': {}}

NEXT Work with data files. TODO in the file replace A, B, C with PLACE, TYPE OF DVELLING, WITH_WHO

In [7]:
at.create_data_file('test_data.csv', ['A', 'B', 'C'], [[1, 2, 3], [11, 22, 33]])

{'id': 'test_data.csv'}

Get generation results

In [8]:
result = at.generate('Complex Plan', data={"A": "CCCC"})
result['variants']

['A house was on the CCCC where he and his brother lived. It was called the old house.']

You can get results for multiple data rows

In [9]:
results = at.generate_bulk('Complex Plan', data=[{"A": "CCC"}, 
                                                 {"A": "AAA"}])
[x['variants'] for x in results]

[['A house was on the CCC where he and his brother lived. It was called the old house.'],
 ['A house was on the AAA where he and his brother lived. It was called the old house.']]

Export State

In [10]:
at.export_state('resources/state.zip')

### Prototype based grammar creation

* Parse non trivial sentence
* Show how it can be abstracted. Going from a single example sentence to more possibilities.
* Go back to JN to emphasise this part - ease of use for the end user - close the presentation.


```clojure
[{:name        "Located"
  :description "Object_N -> Location_N -> Clause"
  :category    "Cl"
  :frames      ["house on the hill"]
  :arguments   [{:name "Object" :category "N" :key "house"}
                {:name "Location" :category "N" :key "hill"}]}]
```

## Limitations

* Parser fails to parse large section of sentences
* Difficulty in naming things (like AMRs and their parameters)
* AMR reusability (maybe)

## Conclusions

TODO